# LegoGears EdgeTPU

- Download LegoGears pretrained
- Change activation to ReLu and run fine-tuning (Edgetpu does not support LeakyReLu)
- Convert to TFLite
- Compile for EdgeTPU

In [ ]:
%env MODEL_DIR=/content/LegoGears_v2_relu

## Install dependencies

In [ ]:
%env DARKNET_VERSION=5.1.82
!if [[ ! -d /content/darknet_tflite ]]; then git clone --depth 1 --branch main https://github.com/sventschui/darknet-tflite.git /content/darknet_tflite; fi
!/content/darknet_tflite/scripts/install_dependencies.sh

# Download dataset

In [ ]:
%%bash

set -e

/content/darknet_tflite/scripts/download_legogears.sh "${MODEL_DIR}"

cd "${MODEL_DIR}"

sed -i "s|/home/stephane/nn/LegoGears|$(pwd)|" LegoGears.data 
sed -i "s|activation=leaky|activation=relu|" LegoGears.cfg 
sed -i "s|batch=1|batch=64|" LegoGears.cfg 

## Fine-tune the model

In [ ]:
!cd "${MODEL_DIR}" && darknet detector train \
  LegoGears.data \
  LegoGears.cfg \
  LegoGears_best.weights \
  -map \
  -dont_show \
  -clear


## Copy pre-existing weights (as an alternative to fine tuning yourself)

In [ ]:
%cp /content/darknet_tflite/weights/LegoGears_best_relu.weights "${MODEL_DIR}/LegoGears_best.weights"

## Test the trained model

In [ ]:
%%bash
cd "${MODEL_DIR}"
set -e
mkdir -p /content/darknet_detections_relu
for f in set_03/*.jpg; do
  darknet detector test LegoGears.data LegoGears.cfg LegoGears_best.weights "$f" -dont_show;
  mv predictions.jpg "/content/darknet_detections_relu/$(basename "$f")";
done

## ONNX Export

In [ ]:
%%bash
set -e
cd "${MODEL_DIR}"
darknet_onnx_export -noboxes LegoGears.cfg LegoGears_best.weights LegoGears.names

### Run inference with ONNX model

In [ ]:
%cd /content
import os
import cv2
import onnxruntime as ort
import matplotlib.pyplot as plt

import functions
functions.reload()

DARKNET_MODEL_BASE_NAME = "LegoGears_v2/LegoGears"
NAMES_FILE = f"{DARKNET_MODEL_BASE_NAME}.names"
CFG_FILE = f"{DARKNET_MODEL_BASE_NAME}.cfg"
CONF_THRESHOLD = 0.5
IOU_THRESHOLD = 0.45
IMAGES_PATH = "LegoGears_v2/set_03"

# Load class names
with open(NAMES_FILE, "r", encoding="utf-8") as f:
    class_names = [line.strip() for line in f if line.strip()]

num_classes = len(class_names)

print(f"✅ Loaded {num_classes} classes")

# Load darknet cfg
net, layers = functions.parse_darknet_cfg(CFG_FILE)
input_width = net["width"]
input_height = net["height"]
yolo_layers = [layer for layer in layers if layer["type"] == "yolo"]

session = ort.InferenceSession(
    f"{DARKNET_MODEL_BASE_NAME}.onnx", providers=["CPUExecutionProvider"]
)

for image_name in [img for img in os.listdir(IMAGES_PATH) if img.endswith(".jpg")]:
    image_path = f"{IMAGES_PATH}/{image_name}"

    print(f"Processing image {image_path}")

    image = cv2.imread(image_path)
    input_tensor, scale, padding, cv_image = functions.preprocess_image(
        input_width, input_height, image
    )

    print("Running inference...")
    input_name = session.get_inputs()[0].name
    outputs = session.run(None, {input_name: input_tensor})

    print("Postprocessing...")
    detections = functions.postprocess_output(
        outputs=outputs,
        yolo_layers_cfg=yolo_layers,
        scale=scale,
        input_size=(input_width, input_height),
        padding=padding,
        class_names=class_names,
        conf_threshold=CONF_THRESHOLD,
        iou_threshold=IOU_THRESHOLD
    )

    print("Visualize...")
    visualized = functions.visualize_detections(
        image=image, detections=detections[0], class_names=class_names
    )

    plt.imshow(cv2.cvtColor(visualized, cv2.COLOR_BGR2RGB), cmap="gray")
    plt.axis("off")
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.show()

## TFLite Conversion

### Generate int8 quantization calibration data

In [ ]:
import cv2
import glob
import numpy as np
import os

import functions
functions.reload()

DARKNET_CONFIG_PATH = f"{os.environ.get("MODEL_DIR")}/LegoGears.cfg"
IMAGES_PATH = f"{os.environ.get("MODEL_DIR")}/set_03"

net, layers = functions.parse_darknet_cfg(DARKNET_CONFIG_PATH)

files = glob.glob(f"{IMAGES_PATH}/*.jpg")
images = []
for idx, file in enumerate(files):
    img = cv2.imread(file)
    img = cv2.resize(img, dsize=(net["width"], net["height"]))
    img = img.astype(np.float32) / 255.0  # convert to 0.0 - 1.0 scale
    images.append(img)

np.save(file=f"{IMAGES_PATH}/calibdata.npy", arr=np.stack(images, axis=0))

### Convert

In [ ]:
!cd "${MODEL_DIR}" && rm -f *.tflite && onnx2tf -i LegoGears.onnx \
    -oiqt \
    -o "${MODEL_DIR}"  \
    -cind "frame" "set_03/calibdata.npy" "[[[[0,0,0]]]]" "[[[[1,1,1]]]]"

### Inference

In [ ]:
import time
import cv2
import os
import numpy as np
from ai_edge_litert.interpreter import Interpreter, load_delegate  # AI Edge Lite / TFLite Runtime
# from tensorflow.lite import Interpreter  # Uncomment if using full TensorFlow instead

import matplotlib.pyplot as plt
%matplotlib inline

import functions
functions.reload()

def load_interpreter(model_path, use_edgetpu=False):
    """
    Load a TensorFlow Lite or EdgeTPU interpreter.
    """
    if use_edgetpu:
        delegates = [load_delegate('libedgetpu.so.1')]
        interpreter = Interpreter(model_path=model_path, experimental_delegates=delegates)
    else:
        interpreter = Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    return interpreter

def dequant(tensor, details):
    if details["dtype"] != np.float32:
        scale_out, zero_point_out = details['quantization']

        return (tensor.astype(np.float32) - zero_point_out) * scale_out
    
    return tensor

def run_inference_tflite(model_path, image, yolo_layers, class_names, use_edgetpu=False):
    """
    Run inference on a single image using a TFLite/AI Edge Lite model.
    """
    # Load the model
    interpreter = load_interpreter(model_path, use_edgetpu)

    # Get input & output details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    input_shape = input_details[0]['shape']
    batch_size, input_height, input_width, channels = input_shape

    # Preprocess image
    input_data, scale, padding, original_image = functions.preprocess_image(input_width, input_height, image)


    # NCHW -> NHWC
    input_data = np.transpose(input_data, (0, 2, 3, 1))

    if input_details[0]["dtype"] != np.float32:
        quant_scale, quant_zero = input_details[0]['quantization']

        input_data = (input_data.astype(np.float32) / quant_scale + quant_zero).round().astype(input_details[0]["dtype"])

        # TODO: Detect output reversion instead of just assuming it for int8
        yolo_layers = list(reversed(yolo_layers))

    # Run inference
    interpreter.set_tensor(input_details[0]['index'], input_data)
    start_time = time.time()
    interpreter.invoke()
    inference_time = (time.time() - start_time) * 1000  # ms


    # Gather outputs (NHWC -> NCHW)
    outputs = [np.transpose(dequant(interpreter.get_tensor(o['index']), o), (0, 3, 1, 2)) for o in output_details]


    # Postprocess    
    print("Postprocessing...")
    detections = functions.postprocess_output(
        outputs,
        yolo_layers_cfg=yolo_layers,
        input_size=(input_width, input_height),
        conf_threshold=CONF_THRESHOLD,
        iou_threshold=IOU_THRESHOLD,
        scale=scale,
        padding=padding,
        class_names=class_names,
    )

    print(f"Inference completed in {inference_time:.2f} ms")
    return detections


NAMES_FILE=f"{os.environ.get("MODEL_DIR")}/LegoGears.names"
CFG_FILE=f"{os.environ.get("MODEL_DIR")}/LegoGears.cfg"
# TF_MODEL=f"{os.environ.get("MODEL_DIR")}/LegoGears_float32.tflite"
TF_MODEL=f"{os.environ.get("MODEL_DIR")}/LegoGears_full_integer_quant.tflite"
CONF_THRESHOLD=0.25
IOU_THRESHOLD=0.45
IMAGES_PATH=f"{os.environ.get("MODEL_DIR")}/set_03"

# Load class names
with open(NAMES_FILE, "r", encoding="utf-8") as f:
    class_names = [line.strip() for line in f if line.strip()]

num_classes = len(class_names)

print(f"✅ Loaded {num_classes} classes")

# Load darknet cfg
net, layers = functions.parse_darknet_cfg(CFG_FILE)
yolo_layers = [layer for layer in layers if layer['type'] == 'yolo']

for image_name in [img for img in os.listdir(IMAGES_PATH) if img.endswith('.jpg')]:
    image_path = f"{IMAGES_PATH}/{image_name}"

    print(f"Processing image {image_path}")

    image = cv2.imread(image_path)

    print("Running inference...")
    detections = run_inference_tflite(TF_MODEL, image=image, 
                                      yolo_layers=yolo_layers,
        class_names=class_names,
                                      
                                      use_edgetpu=False)


    print("Visualize...")
    visualized = functions.visualize_detections(
        image=image,
        detections=detections[0],
        class_names=class_names
    )

    plt.imshow(cv2.cvtColor(visualized, cv2.COLOR_BGR2RGB),cmap='gray')
    plt.axis("off")
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    plt.show()




## Compile for edgetpu

In [ ]:
%env EDGETPU_COMPILER_MODEL_INPUT=LegoGears_full_integer_quant.tflite
!cd "${MODEL_DIR}" && edgetpu_compiler "${EDGETPU_COMPILER_MODEL_INPUT}" --show_operations